This notebook is for snapping of substations to lines

In [1]:
import os
import matplotlib.pyplot as plt 
import geopandas as gpd
#import geoplot
import pandas as pd
import numpy as np
from shapely.geometry import LineString
from shapely.geometry import Point

# Africa shape data

In [2]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
africa = world.query('continent == "Africa"')
nigeriaplot = world.query('name == "Nigeria"')
#world

# Load data

In [3]:
# pathg = os.path.realpath("data")+'/africa_all_generators.geojson' # Generators are not required in base_network
paths = os.path.realpath("data/clean")+'/africa_all_buses_clean.geojson'
pathl = os.path.realpath("data/clean")+'/africa_all_lines_clean.geojson'

# generators = gpd.read_file(pathg).set_crs(epsg=4326, inplace=True)
substations = gpd.read_file(paths).set_crs(epsg=4326, inplace=True)
lines = gpd.read_file(pathl).set_crs(epsg=4326, inplace=True)

In [5]:
# Filter only Nigeria
lines_ng = lines[lines.loc[:,"country"] == "nigeria"].copy()
substations_ng = substations[substations.loc[:,"country"] == "nigeria"].copy()

In [6]:
# Assign to every line a start and end point

lines_ng["bounds"] = lines_ng["geometry"].boundary # create start and end point
# splits into coordinates
lines_ng["bus0_lon"] = lines_ng["bounds"].bounds.iloc[:,0]
lines_ng["bus0_lat"] = lines_ng["bounds"].bounds.iloc[:,1]
lines_ng["bus1_lon"] = lines_ng["bounds"].bounds.iloc[:,2]
lines_ng["bus1_lat"] = lines_ng["bounds"].bounds.iloc[:,3]

lines_ng['bus_0_coors']=lines_ng["bounds"].apply(lambda mp: mp[0])
lines_ng['bus_1_coors']=lines_ng["bounds"].apply(lambda mp: mp[1])

In [10]:
#lines_ng
#substations_ng